In [1]:
import numpy as np
import astropy.io.fits as pf
import math
import matplotlib.pyplot as plt

In [5]:
# Загружаем данные с красными смещениями
data = np.genfromtxt('goodss_3dhst.v4.1.zout', skip_header=2)

z_spec = data[:,1]
z_a  = data[:,2]
numb = data[:,0]

# Запишем условие на обьекты
filt = (z_a >= 6)

# Отберём нужные
z_a1 = z_a[filt]
z_spec1 = z_spec[filt]
numb1 = numb[filt]

In [6]:
# Загружаем координаты обьектов из главного каталога
XX = np.genfromtxt('goodss_3dhst.v4.1.cat')[:, 1]
YY = np.genfromtxt('goodss_3dhst.v4.1.cat')[:, 2]


# Отберём нужные
X1 = XX[filt]
Y1 = YY[filt]


# Первая проверка
print(len(X1), len(z_a1))

1969 1969


In [74]:
# Из выбранных обьектов отбираем те, которые находятся в нужной области и формируем новые массивы координат, 
# одновременно с новыми массивами для красного смещения и номерами обьектов
X0 = []
Y0 = []
rs = []
n =  []
for i in range (len(X1)):

    if (0 < X1[i] - 6000 < 4000) and (0 < Y1[i] - 10000 < 4000):
        X0.append(X1[i] - 6000)
        Y0.append(Y1[i] - 10000)
        rs.append(z_a1[i])
        n.append(numb1[i])
        


In [17]:
# Вторая проверка
print(len(rs),len(X0),len(Y0),len(n))
print(type(X0),type(Y0),type(rs))


575 575 575 575
<class 'list'> <class 'list'> <class 'list'>


In [194]:
# Третья проверка для алгоритма расстояний
k = np.array([3.99,4.222,5.99,6.000])
As = np.zeros((len(k),len(k)))
tempk = np.zeros(len(k))


for j in range (len(k)):
     if (j % 2 == 0):
         print('wait')    
     tempk = abs(k - k[j])
     As[j,:] = tempk
        
print(As)

wait
wait
[[ 0.     0.232  2.     2.01 ]
 [ 0.232  0.     1.768  1.778]
 [ 2.     1.768  0.     0.01 ]
 [ 2.01   1.778  0.01   0.   ]]


In [20]:
# Создаём матрицы расстояний для координат и красного смещения
Ax = np.zeros((len(X0),len(X0)))
Ay = np.zeros((len(Y0),len(Y0)))
Az = np.zeros((len(rs),len(rs)))

tx = np.zeros(len(X0))
ty = np.zeros(len(Y0))
tz = np.zeros(len(rs))
for j in range (len(X0)):   
     tx = abs(X0 - X0[j])
     Ax[j,:] = tx
     ty = abs(Y0 - Y0[j])
     Ay[j,:] = ty
     tz = abs(rs - rs[j])
     Az[j,:] = tz
  

In [21]:
# Четвёртая проверка
print('X')
print(Ax)
print('Y')
print(Ay)
print('Z')
print(Az)

X
[[    0.     1082.439  1871.447 ...,  1285.84   1783.999  2931.684]
 [ 1082.439     0.      789.008 ...,   203.401   701.56   1849.245]
 [ 1871.447   789.008     0.    ...,   585.607    87.448  1060.237]
 ..., 
 [ 1285.84    203.401   585.607 ...,     0.      498.159  1645.844]
 [ 1783.999   701.56     87.448 ...,   498.159     0.     1147.685]
 [ 2931.684  1849.245  1060.237 ...,  1645.844  1147.685     0.   ]]
Y
[[  0.00000000e+00   4.07060000e+01   1.46556000e+02 ...,   3.89457600e+03
    3.89664100e+03   3.94368700e+03]
 [  4.07060000e+01   0.00000000e+00   1.05850000e+02 ...,   3.85387000e+03
    3.85593500e+03   3.90298100e+03]
 [  1.46556000e+02   1.05850000e+02   0.00000000e+00 ...,   3.74802000e+03
    3.75008500e+03   3.79713100e+03]
 ..., 
 [  3.89457600e+03   3.85387000e+03   3.74802000e+03 ...,   0.00000000e+00
    2.06500000e+00   4.91110000e+01]
 [  3.89664100e+03   3.85593500e+03   3.75008500e+03 ...,   2.06500000e+00
    0.00000000e+00   4.70460000e+01]
 [  3.9436870

In [98]:
# Смотрим сколько обьектов находятся в пределах 2.5 пикселя от 15-го. 
G = []
G = Ax[15,:]
k = 0
for i in range (len(G)):
    if (0 < G[i] < 2.5):
        k+=1
print(k)        

4


In [99]:
# Ищем сколько в каждой строке обьектов, которые находятся близко друг другу и
# имеют одинаковые красные смещения

candidatsx = [np.sum((Ax[i] > 0) & (Ax[i] < 2.5)) for i in range(len(X0))]
print(candidatsx)
print()

candidatsy = [np.sum((Ay[i] > 0) & (Ay[i] < 2.5)) for i in range(len(Y0))]



# Во избежании тех случаев когда разница берётся от одного обьекта
for i in range (len(Az)):
    for j in range (len(Az)):
        if i == j :
            Az[i,j] = 1000
            
        
candidatsz = [np.sum((Az[i] >= 0) & (Az[i] < 0.0000000000000000011)) for i in range(len(rs))]
print(candidatsz)




[0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 3, 0, 3, 3, 0, 4, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 1, 1, 2, 1, 2, 1, 0, 0, 1, 0, 2, 0, 1, 0, 0, 1, 0, 2, 0, 1, 0, 1, 3, 1, 3, 2, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 2, 1, 1, 0, 1, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 3, 1, 2, 1, 1, 3, 1, 1, 0, 1, 2, 0, 2, 1, 1, 4, 3, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 2, 1, 2, 1, 1, 2, 0, 3, 4, 1, 3, 0, 2, 2, 0, 0, 1, 0, 4, 0, 0, 0, 3, 4, 0, 2, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 3, 0, 0, 2, 1, 1, 0, 1, 3, 4, 0, 0, 1, 3, 0, 3, 1, 1, 2, 0, 2, 1, 1, 1, 0, 1, 2, 1, 2, 1, 1, 0, 3, 0, 0, 3, 1, 1, 0, 0, 2, 4, 2, 1, 1, 0, 1, 1, 2, 2, 2, 2, 3, 2, 0, 0, 2, 1, 3, 0, 1, 1, 0, 0, 0, 4, 2, 2, 3, 0, 0, 2, 1, 0, 0, 0, 2, 2, 0, 0, 2, 1, 1, 0, 1, 0, 1, 3, 0, 1, 1, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 1, 3, 2, 0, 0, 0, 1, 0, 1, 0, 1, 2, 2, 1, 1, 1, 2, 0, 2, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 2, 0, 2, 2, 0, 1, 0, 3, 1, 0, 2, 0, 1, 0, 1, 0, 0, 1, 1, 

In [96]:
# Из столбцов координат и красного смещения выбираем обьекты где выполняются все три критерия
X0 = np.array(X0)
x = X0[np.where((np.array(candidatsx)>0) & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]
print(len(x))
print([np.where((np.array(candidatsx)>0) & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]])
print()

Y0 = np.array(Y0)
y = Y0[np.where( (np.array(candidatsx)>0)  & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]
print(y)
print()

rs = np.array(rs)
z = rs[np.where((np.array(candidatsx)>0) & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]
print(z)

n = np.array(n)
m = n[np.where((np.array(candidatsx)>0) & (np.array(candidatsy)>0)  & (np.array(candidatsz)>0) )[0]]



235
[array([  4,  28,  38,  42,  45,  47,  51,  52,  54,  55,  59,  60,  61,
        63,  66,  72,  74,  76,  77,  81,  83,  86,  92,  98,  99, 103,
       104, 106, 108, 110, 112, 113, 115, 116, 122, 123, 125, 127, 128,
       129, 132, 133, 134, 141, 143, 147, 150, 151, 153, 155, 156, 157,
       158, 159, 161, 164, 167, 168, 171, 172, 173, 175, 176, 177, 180,
       181, 183, 184, 185, 186, 188, 190, 191, 193, 194, 195, 197, 198,
       210, 211, 214, 215, 216, 218, 224, 225, 226, 228, 229, 234, 235,
       236, 239, 240, 248, 249, 250, 252, 254, 255, 257, 260, 264, 269,
       270, 275, 279, 280, 281, 282, 283, 284, 285, 287, 289, 290, 291,
       292, 301, 303, 304, 305, 307, 309, 312, 317, 319, 321, 322, 324,
       326, 328, 331, 332, 333, 335, 336, 337, 340, 341, 342, 346, 349,
       350, 352, 353, 354, 355, 356, 359, 366, 368, 369, 372, 375, 376,
       377, 381, 387, 388, 389, 392, 393, 394, 395, 397, 399, 403, 404,
       407, 410, 413, 414, 419, 424, 425, 426, 429, 433, 43

In [93]:
# Пятая проверка 
A = np.array([[1.3,2,3,8,9],[4,5,6,8,9],[1,1,1,9,9],[1,9,9,9,7],[3,2,3,4,5]])
candidats1 = [np.sum((A[i]>0) & (A[i] < 2)) for i in range(5)]
print(candidats1)
print()

B = np.array([[1.8,2,3,6,6],[4,5,6,8,6],[4,4,4,9,1],[1,0,9,9,3],[3,3,3,3,3]])
candidats2 = [np.sum((B[i]>0) & (B[i] < 2)) for i in range(5)]
print(candidats2)
print()

C = np.array([[1,2,3,1,4],[4,5,6,8,4],[1,1,1,9,4],[1.0,9,9,9.0,4],[1,4,4,4,4]])
candidats3 = [np.sum((C[i]>0) & (C[i] < 2)) for i in range(5)]
print(candidats3)
print()

g=np.array([11.4,21,31,41,91])
g[np.where((np.array(candidats1)>0) & (np.array(candidats2)>0)  & (np.array(candidats3)>0) )[0]]


[1, 0, 3, 1, 0]

[1, 0, 1, 1, 0]

[2, 0, 3, 1, 1]



array([ 11.4,  31. ,  41. ])

In [94]:
# Составляем таблицу парных обьектов
Yes = np.vstack((m, x, y, z))
np.savetxt('intr.dat', np.transpose(Yes), delimiter="  ", fmt="%s")

In [89]:
dat = np.genfromtxt('intr.dat')

x = dat[:,1]
y = dat[:,2]
z = dat[:,3]

In [97]:
# Смотрим что получилось
print(x)
print()
print(y)
print()
print(z)



[ 1890.279  2412.595  1847.233  1976.355  2383.263  2706.181  2683.688
  2379.759  2572.512  2319.53   3230.426  2683.477  2717.717  2865.121
  3232.293  2707.478  1975.541  2468.975  3081.175  2928.329  2808.675
  3423.553  2610.95   1772.009  2262.227  2285.806  2677.303  2187.833
  2970.235  2708.609  2527.918  2867.038  3179.613  1911.071  3181.561
  1807.084  3039.426  1867.846  2424.094  2172.881  2751.776  2074.046
  2943.064  3125.356  2403.813  2402.525  2907.912  3269.101  2160.732
  2973.986  2418.271  1986.187  1614.189  3354.608  3066.042  2107.084
  2107.283  2019.077  2047.263  2172.418  2205.99   3005.92   2262.34
  2112.089  2233.588  2571.926  1996.622  3354.943  3003.382  1421.006
  2927.807  1413.702  1911.413  2895.112  2806.789  1397.156  2188.304
  2835.242  2981.518  1843.334  2056.562  3444.571  1401.896  1794.911
  1846.888  3563.851  2262.146  3127.1    2332.784  2906.459  2237.997
  1864.962  2939.258  2976.02   2676.405  1671.041  2506.939  2272.597
  1375.